# What this script does

We create a dataset of all deficiencies committed by WA-based nursing homes from 2017.

# I. SETTINGS

In [1]:
import pandas as pd
from os import listdir
import re

# II. IMPORT DATA

## Deficienies (CMS)

In [3]:
# Obtain a list of all the downloaded adult family PDF reports
source_path = '../../covid19_nursing_homes_big_data/Full-Statement-of-Deficiencies-July-2020/'
file_list = listdir(source_path)

# Weed out any files in the folder that are not PDFs
file_list = [file for file in file_list if re.search('\.xlsx$', file)]
file_list = pd.Series(file_list)

df_sod_orig = pd.DataFrame()

for file in file_list:
    print(file)
    df_temp = pd.read_excel(source_path + file,
                            header=0, 
                            usecols=range(0,13), 
                            dtype={'facility_id':object, 'zip':object, 'deficiency_tag':object})
    df_sod_orig = pd.concat([df_sod_orig, df_temp])
    del(df_temp)

df_sod_orig = df_sod_orig.reset_index(drop=True)

# Save down a CSV version
df_sod_orig.to_csv('../../covid19_nursing_homes_big_data/cms_sod_txt.csv', index=False)

text2567_20200701_cms_reg6.xlsx
text2567_20200701_cms_reg7.xlsx
text2567_20200701_cms_reg10.xlsx
text2567_20200701_cms_reg5a.xlsx
text2567_20200701_cms_reg1.xlsx
text2567_20200701_cms_reg2.xlsx
text2567_20200701_cms_reg3.xlsx
text2567_20200701_cms_reg5b.xlsx
text2567_20200701_cms_reg8.xlsx
text2567_20200701_cms_reg4.xlsx
text2567_20200701_cms_reg9.xlsx


In [4]:
df_sod_wa = pd.read_csv('/Users/mvilla/Documents/repos/covid19_nursing_homes_big_data/cms_WA/cms_WA.csv')

In [6]:
df_sod_orig.head()

,facility_name,facility_id,address,city,state,zip,inspection_date,deficiency_tag,scope_severity,complaint,standard,eventid,inspection_text
0,ROGERS HEALTH AND REHABILITATION CENTER,045070,1149 W NEW HOPE RD,ROGERS,AR,72758,04/25/2018,0677,D,1,0,RHC611,**NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO ...
1,ROGERS HEALTH AND REHABILITATION CENTER,045070,1149 W NEW HOPE RD,ROGERS,AR,72758,04/25/2018,0688,D,1,0,RHC611,**NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO ...
2,ROGERS HEALTH AND REHABILITATION CENTER,045070,1149 W NEW HOPE RD,ROGERS,AR,72758,04/25/2018,0791,D,1,0,RHC611,**NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO ...
3,ROGERS HEALTH AND REHABILITATION CENTER,045070,1149 W NEW HOPE RD,ROGERS,AR,72758,05/05/2017,0333,J,1,0,193I11,**NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO ...
4,ROGERS HEALTH AND REHABILITATION CENTER,045070,1149 W NEW HOPE RD,ROGERS,AR,72758,06/28/2017,0153,B,1,0,NKZK11,**NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO ...


In [5]:
df_sod_wa.head()

,facility_name,facility_id,address,city,state,zip,inspection_date,deficiency_tag,scope_severity,complaint,standard,eventid,inspection_text,filedate,cms_region
0,ISSAQUAH NURSING AND REHABILITATION CENTER,505004,805 FRONT STREET,ISSAQUAH,WA,98027,2017-04-26,514,D,1.0,0.0,DY4R11,**NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO ...,2020-06-01,10
1,ISSAQUAH NURSING AND REHABILITATION CENTER,505004,805 FRONT STREET,ISSAQUAH,WA,98027,2017-08-29,253,E,0.0,1.0,PWON11,"<BR/>Based on observation and interview, the f...",2020-06-01,10
2,ISSAQUAH NURSING AND REHABILITATION CENTER,505004,805 FRONT STREET,ISSAQUAH,WA,98027,2017-08-29,279,D,0.0,1.0,PWON11,**NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO ...,2020-06-01,10
3,ISSAQUAH NURSING AND REHABILITATION CENTER,505004,805 FRONT STREET,ISSAQUAH,WA,98027,2017-08-29,328,D,0.0,1.0,PWON11,**NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO ...,2020-06-01,10
4,ISSAQUAH NURSING AND REHABILITATION CENTER,505004,805 FRONT STREET,ISSAQUAH,WA,98027,2017-08-29,329,D,0.0,1.0,PWON11,**NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO ...,2020-06-01,10


## F-Tags (CMS)

The SOD dataframe above contains tha tag code for each defficiency recorded, but it doesn't contain the general group that each of those tag belongs to. The information is containd in [this list of the revised F-tags](https://www.cms.gov/Medicare/Provider-Enrollment-and-Certification/GuidanceforLawsAndRegulations/Downloads/List-of-Revised-FTags.pdf). 

A version is in this [F-Tag crosswalk Excel file](https://www.cms.gov/Medicare/Provider-Enrollment-and-Certification/GuidanceforLawsAndRegulations/Downloads/F-Tag-Crosswalk.xlsx). This is the data we are importing now and that will be adding to the SOD dataset later in the script:

In [ ]:
df_tags_orig = pd.read_excel('../A_source_data/CMS/LTC FTags_Phase 2_Crosswalk.xlsx',
                             sheet_name='Sortable by Tags', usecols='A:H')
df_tags_orig.columns = ['tag', 'sqc_tag?', 'tag_title', 'cfr', 'tag_group', 'phase3', 'tag_old', 'moved_text']

So now we have a data frame that contains all the deficiencies found in all surveys carried out, and another dataframe that contains detailed information about the tags used to classify those deficiencies. We need to join both dataframes.

In [ ]:
df_sod_orig.columns

## Severity code descriptions

The SOD dataframe also contains codes for the severity of each deficiency, but not a description of the severity level of each of those codes. Those descriptions can be found in the docment [Design for Nursing Home Compare
Five-Star Quality Rating System:
Technical Users’ Guide](https://www.cms.gov/Medicare/Provider-Enrollment-and-Certification/CertificationandComplianc/downloads/usersguide.pdf). The following mapping is based on that document:

In [ ]:
severity = [['A', 'No actual harm with potential for minimal harm - Isolated'],
            ['B', 'No actual harm with potential for minimal harm - Pattern'],
            ['C', 'No actual harm with potential for minimal harm - Widespread'],
            ['D', 'No actual harm with potential for more than minimal harm that is not immediate jeopardy - Isolated'],
            ['E', 'No actual harm with potential for more than minimal harm that is not immediate jeopardy - Pattern'],
            ['F', 'No actual harm with potential for more than minimal harm that is not immediate jeopardy - Widespread'],
            ['G', 'Actual harm that is not immediate jeopardy - Isolated'],
            ['H', 'Actual harm that is not immediate jeopardy - Pattern'],
            ['I', 'Actual harm that is not immediate jeopardy - Widespread'],
            ['J', 'Immediate jeopardy to resident health or safety - Isolated'],
            ['K', 'Immediate jeopardy to resident health or safety - Pattern'],
            ['L', 'Immediate jeopardy to resident health or safety - Widespread']]

severity = pd.DataFrame(severity, columns=['scope_severity', 'severity_desc'])
severity

# Consitency test
assert set(df_sod_orig['scope_severity']).issubset(set(severity['scope_severity']))

# III. REDUCING: WA STATE

In [ ]:
# del(df_sod_wa)
df_sod_wa = df_sod_orig.copy()
print(df_sod_wa.shape)

# Reduce to only WA homes
df_sod_wa = df_sod_wa[df_sod_wa['state'] == 'WA']
print(df_sod_wa.shape)

# Add the severity descriptions
df_sod_wa = df_sod_wa.join(severity.set_index('scope_severity'), on='scope_severity', how='left')

# Create a proper date column
df_sod_wa['inspection_dt'] = pd.to_datetime(df_sod_wa['inspection_date'])

# Eliminate unnecesary fields and reset index
df_sod_wa = df_sod_wa.drop(['address', 'city', 'state', 'zip', 'inspection_date', 'inspection_text'], axis=1)
df_sod_wa = df_sod_wa.drop_duplicates().reset_index(drop=True)

# Change some column names into something easier to use
df_sod_wa = df_sod_wa.rename(columns={'deficiency_tag':'tag', 
                                      'scope_severity':'severity_code'})

print(df_sod_wa.shape)

In [ ]:
df_sod_wa

#### What does each row this dataframe represent?

Question: Is each of the 11438 rows a unique combination of facility/survey/deficiency?

In [ ]:
temp = df_sod_wa[['facility_id', 'eventid', 'tag']]
assert len(temp.drop_duplicates()) == len(df_sod_wa)
del(temp)

That is indeed the case. So each row seem to represent:
- a single deficiency
- found at a particular facility
- during a particular event (i.e., inspection or investigation)

In [ ]:
df_sod_wa.nunique()

#### What is the tiime period covered by the dataset?

In [ ]:
print(df_sod_wa['inspection_dt'].min())
print(df_sod_wa['inspection_dt'].max())

# IV. JOINING BOTH DATAFAMES

In [ ]:
df_tags = df_tags_orig.copy()

In [ ]:
# df_sod_wa
print('There are', df_sod_wa['tag'].nunique(), 'different tags in df_sod_wa\n')

# df_tags
print('There are', df_tags['tag'].nunique(), 'different NEW tags in df_tags')
print('There are', df_tags['tag_old'].nunique(), 'different OLD tags in df_tags')

## Building a mapping table via tag numbers

First of all, based on a comparison of the two documents mentioned earlier ([the list of the revised F-tags](https://www.cms.gov/Medicare/Provider-Enrollment-and-Certification/GuidanceforLawsAndRegulations/Downloads/List-of-Revised-FTags.pdf) and the [F-Tag crosswalk spreadsheet](https://www.cms.gov/Medicare/Provider-Enrollment-and-Certification/GuidanceforLawsAndRegulations/Downloads/F-Tag-Crosswalk.xlsx)) there are a few adjustments that need to be made.

In [ ]:
df_tags['tag_group'] = df_tags['tag_group'].str.replace('483.20  Resident Assessments\n483.70  Administration', '483.70  Administration')
df_tags['tag_group'] = df_tags['tag_group'].str.replace('483.10 Resident Rights\n483.12  Freedom from Abuse, Neglect, and Exploitation', '483.12  Freedom from Abuse, Neglect, and Exploitation')
df_tags['tag_group'] = df_tags['tag_group'].str.replace('483.10 Resident Rights\n483.90  Physical Environment', '483.90  Physical Environment')

Now we split the tag group numbers and names.

In [ ]:
df_tags['tag_group_num'] = df_tags['tag_group'].str.extract('(\d+\.\d+)')
df_tags['tag_group_name'] = df_tags['tag_group'].str.replace('\d+\.\d+', '').str.strip()

In [ ]:
df_tags

In [ ]:
# New tags
df_tags_new = pd.DataFrame(df_tags[['tag', 'tag_group_num', 'tag_group_name']])
df_tags_new['tag_old_new'] = 'New'

# Old tags
df_tags_old = pd.DataFrame(df_tags[['tag_old', 'tag_group_num', 'tag_group_name']])
df_tags_old = df_tags_old.rename(columns={'tag_old':'tag'})
df_tags_old['tag_old_new'] = 'Old'

# Old and new together
df_tag_map = pd.concat([df_tags_new, df_tags_old], axis=0)

# Reduce and tidy up
df_tag_map = df_tag_map.dropna(axis=0, how='any')
df_tag_map = df_tag_map.drop_duplicates()
df_tag_map = df_tag_map.sort_values(['tag', 'tag_group_num'], ascending=True)
df_tag_map = df_tag_map.reset_index(drop=True)

In [ ]:
df_tag_map

In [ ]:
df_tag_map['tag_group_name'].value_counts()

#### Check and adjust for double classifications

Back in 2018, some tags were renamed and/or reclassified. The mapping table below addressses that reclassification, except for one situation: When a tag is not fully "transported" into another section, but broken down into components and then those components being moved around. 

For example, the old tag F309 was belonged to the *Quality of Life* regulatory grouping. When F309 was reviewed, it got broken down into components and one of them was reclassified into the *Behavioral Health Services* group. So now when we do the mapping from old to new tags, each instance of F309 produces two records that are copies of each other, except for the grouping. In other words, we are double counting.

Here we adjust to correct that.

In [ ]:
# If a tag show up more than once in the mapping table, 
# it means it was wrongly assigned more than tag group. Let's find those.
double_count = df_tag_map['tag'].value_counts()
double_count = double_count[double_count > 1].reset_index()
double_count = df_tag_map[df_tag_map['tag'].isin(double_count['index'])]

double_count

We found [an old CMS document](https://www.cms.gov/Regulations-and-Guidance/Guidance/Transmittals/downloads/R5SOM.pdf), that sheds light into the previous grouping of the old codes. The following adjustments are made based on that document:

In [ ]:
df_tag_map.loc[df_tag_map['tag']=='F155', ['tag_group_num','tag_group_name']] = ['483.10','Resident Rights']
df_tag_map.loc[df_tag_map['tag']=='F164', ['tag_group_num','tag_group_name']] = ['483.10','Resident Rights']
df_tag_map.loc[df_tag_map['tag']=='F280', ['tag_group_num','tag_group_name']] = ['483.10','Resident Rights']
df_tag_map.loc[df_tag_map['tag']=='F226', ['tag_group_num','tag_group_name']] = ['483.12','Freedom from Abuse, Neglect, and Exploitation']
df_tag_map.loc[df_tag_map['tag']=='F279', ['tag_group_num','tag_group_name']] = ['483.21','Comprehensive Resident Centered Care Plans']
df_tag_map.loc[df_tag_map['tag']=='F309', ['tag_group_num','tag_group_name']] = ['483.25','Quality of Care']
df_tag_map.loc[df_tag_map['tag']=='F461', ['tag_group_num','tag_group_name']] = ['483.90','Physical Environment']
df_tag_map.loc[df_tag_map['tag']=='F498', ['tag_group_num','tag_group_name']] = ['483.35','Nursing Services']

df_tag_map = df_tag_map.drop_duplicates().reset_index(drop=True)

In [ ]:
df_tag_map

In [ ]:
# After our reclassification, let's see if there remain any tags that are assigned to more than group.
double_count = df_tag_map['tag'].value_counts()
double_count = double_count[double_count > 1].reset_index()
double_count = df_tag_map[df_tag_map['tag'].isin(double_count['index'])]
double_count = double_count.drop_duplicates()

double_count

Just one. We checked, and it is not present in the ***df_sod_wa***, so we let it be.

Now we standardize some of the names of the tag_group_name, to make them consistent with the way they are named in the WA state regulation

In [ ]:
df_tag_map['tag_group_name'] = df_tag_map['tag_group_name'].str.replace('Resident Rights.*', 'Resident Rights')
df_tag_map['tag_group_name'] = df_tag_map['tag_group_name'].str.replace('Admission, Transfer, and Discharge', 'Admission, Transfer and Discharge')
df_tag_map['tag_group_name'] = df_tag_map['tag_group_name'].str.replace('Resident Assessments.*', 'Resident Assessment and Plan of Care')
df_tag_map['tag_group_name'] = df_tag_map['tag_group_name'].str.replace('Specialized Rehabilitative Services', 'Specialized Habilitative and Rehabilitative Services')
df_tag_map['tag_group_name'] = df_tag_map['tag_group_name'].str.replace('Food and Nutrition Services', 'Food Services Areas')

# Reduce and tidy up
df_tag_map = df_tag_map.drop_duplicates().reset_index(drop=True)

In [ ]:
df_tag_map

In [ ]:
df_tag_map['tag_group_name'].value_counts(dropna=False)

## Joining

We now have a mapping table ready.
Now let's make sure that the tag number columns in each of the two datasets we will join have consistent values.

In [ ]:
print('Unique tags in df_sod_wa =', df_sod_wa['tag'].nunique())
print(df_sod_wa['tag'].unique())
print('\r')

print('Unique tags in df_tag_map =', df_tag_map['tag'].nunique())
print(df_tag_map['tag'].unique())

In *df_sod_wa*, instead of an 'F', the tags start with a '0'. Here we make them compatible.

In [ ]:
df_tag_map['tag'] = df_tag_map['tag'].str.replace('F', '0')
df_tag_map['tag'] = df_tag_map['tag'].str.replace('\n', '')

print(len(set(df_sod_wa['tag']).intersection(set(df_tag_map['tag']))))

Before

In [ ]:
df_sod_wa

In [ ]:
df_sod_wa = df_sod_wa.join(df_tag_map.set_index('tag'), on='tag', how='left')

After

In [ ]:
df_sod_wa

Reorder columns

In [ ]:
df_sod_wa = df_sod_wa[['facility_name', 'facility_id',
                       'eventid', 'inspection_dt', 
                       'tag', 'tag_group_num', 'tag_group_name', 'tag_old_new', 
                       'severity_code', 'severity_desc',
                       'complaint', 'standard']]

# V. EXPORTING RESULTS

In [ ]:
df_sod_wa.to_csv('../C_output_data/sod_wa.csv', index=False)